In [30]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from fbprophet import Prophet
mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#from sktime.forecasting.model_selection import temporal_train_test_split
#from sktime.utils.plotting.forecasting import plot_ys
%matplotlib inline



In [31]:
df=pd.read_csv('/Users/kasturid3/Desktop/vaccines_by_age.csv',parse_dates=['Date'])

In [32]:
df

,Date,Agegroup,At least one dose_cumulative,Second_dose_cumulative,fully_vaccinated_cumulative,third_dose_cumulative,Total population,Percent_at_least_one_dose,Percent_fully_vaccinated,Percent_3doses
0,2020-12-16,12-17yrs,0,0.0,NaN,NaN,951519,0.0000,0.0000,NaN
1,2020-12-16,18-29yrs,45,0.0,NaN,NaN,2455535,0.0000,0.0000,NaN
2,2020-12-16,30-39yrs,66,0.0,NaN,NaN,2056059,0.0000,0.0000,NaN
3,2020-12-16,40-49yrs,98,0.0,NaN,NaN,1876583,0.0000,0.0000,NaN
4,2020-12-16,50-59yrs,141,0.0,NaN,NaN,2060934,0.0000,0.0000,NaN
...,...,...,...,...,...,...,...,...,...,...
5418,2022-03-31,80,675107,NaN,659945.0,580715.0,655835,0.9999,0.9999,0.8855
5419,2022-03-31,Adults_18plus,11140351,NaN,10873399.0,7045917.0,11971129,0.9306,0.9083,0.5886
5420,2022-03-31,Ontario_12plus,12004141,NaN,11759405.0,7179239.0,12932471,0.9282,0.9093,0.5551
5421,2022-03-31,Ontario_5plus,12603706,NaN,12116728.0,7179322.0,14010998,0.8996,0.8648,0.5124


In [33]:
df=df[['Date','Agegroup','At least one dose_cumulative']]

In [34]:
df=df[df['Date']>'2021-06-01'].reset_index(drop=True)

In [35]:
df1=df.pivot_table(index=['Date'],columns='Agegroup',values=['At least one dose_cumulative']).reset_index()

In [36]:
df1.columns=['Date','05-11yrs','12-17yrs','18-29yrs','30-39yrs','40-49yrs','50-59yrs','60-69yrs','70-79yrs','80','Adults_18plus','Ontario_12plus','Ontario_5plus','Undisclosed_or_missing']

In [37]:
df2=df1[['Date','12-17yrs', '18-29yrs', '30-39yrs', '40-49yrs',
       '50-59yrs', '60-69yrs', '70-79yrs', '80','Adults_18plus', 'Ontario_12plus']]

In [38]:
df2

,Date,12-17yrs,18-29yrs,30-39yrs,40-49yrs,50-59yrs,60-69yrs,70-79yrs,80,Adults_18plus,Ontario_12plus
0,2021-06-02,230283.0,1221667.0,1152765.0,1219742.0,1512059.0,1523603.0,1054369.0,641700.0,8325905.0,8556188.0
1,2021-06-03,248847.0,1250368.0,1173607.0,1234987.0,1520527.0,1527039.0,1055612.0,642254.0,8404394.0,8653241.0
2,2021-06-04,269495.0,1281318.0,1196845.0,1251085.0,1530707.0,1531016.0,1057146.0,642865.0,8490982.0,8760477.0
3,2021-06-05,288985.0,1312634.0,1220621.0,1267057.0,1540128.0,1534777.0,1058463.0,643437.0,8577117.0,8866102.0
4,2021-06-06,307419.0,1340248.0,1240020.0,1279695.0,1547643.0,1537789.0,1059540.0,643861.0,8648796.0,8956215.0
...,...,...,...,...,...,...,...,...,...,...,...
298,2022-03-27,863538.0,2212778.0,1845798.0,1684111.0,1858694.0,1721220.0,1140408.0,675072.0,11138081.0,12001619.0
299,2022-03-28,863576.0,2212877.0,1845861.0,1684148.0,1858728.0,1721242.0,1140422.0,675075.0,11138353.0,12001929.0
300,2022-03-29,863623.0,2213077.0,1846012.0,1684207.0,1858799.0,1721279.0,1140444.0,675088.0,11138906.0,12002529.0
301,2022-03-30,863704.0,2213362.0,1846164.0,1684306.0,1858894.0,1721358.0,1140475.0,675096.0,11139655.0,12003359.0


In [39]:
df2=df2.set_index('Date')



In [40]:
df2.index = pd.to_datetime(df2.index)

In [41]:
df2.index=df2.index.to_period("D")


In [42]:
y = pd.Series(df2['12-17yrs'])

In [43]:
train=df2['12-17yrs'][:280]
test=df2['12-17yrs'][280:]

In [44]:
test

Date
2022-03-09    861468.0
2022-03-10    861617.0
2022-03-11    861733.0
2022-03-12    861879.0
2022-03-13    862044.0
2022-03-14    862098.0
2022-03-15    862226.0
2022-03-16    862355.0
2022-03-17    862509.0
2022-03-18    862660.0
2022-03-19    862812.0
2022-03-20    862926.0
2022-03-21    862978.0
2022-03-22    863055.0
2022-03-23    863157.0
2022-03-24    863223.0
2022-03-25    863319.0
2022-03-26    863427.0
2022-03-27    863538.0
2022-03-28    863576.0
2022-03-29    863623.0
2022-03-30    863704.0
2022-03-31    863790.0
Freq: D, Name: 12-17yrs, dtype: float64

In [28]:
from statsmodels.tsa.arima_model import ARIMA

# 1,1,2 ARIMA Model
model = ARIMA(train, order=(1,1,2))
model=model.fit()
pred=model.predict(start=1, end=24, exog=None, dynamic=False)


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 re

In [29]:
pred

2021-06-03      864.347909
2021-06-04    18369.771746
2021-06-05    19591.993838
2021-06-06    19558.926732
2021-06-07    19070.048390
2021-06-08    17365.879850
2021-06-09    16897.773946
2021-06-10    17173.358767
2021-06-11    16836.124034
2021-06-12    16388.226721
2021-06-13    14871.047048
2021-06-14    14226.238294
2021-06-15    12186.847111
2021-06-16    11592.613360
2021-06-17    11902.873990
2021-06-18    11478.596588
2021-06-19    10688.597153
2021-06-20     9722.139433
2021-06-21     9810.855259
2021-06-22     8013.925421
2021-06-23     7051.878522
2021-06-24     6825.418123
2021-06-25     6428.158123
2021-06-26     6071.535596
Freq: D, dtype: float64

In [22]:
rmse = np.mean((y_pred - y_test)**2)**.5  # RMSE
rmse

616.7426352721978

In [23]:
y_pred

2022-03-08    861285.766691
2022-03-09    861360.435247
2022-03-10    861435.107283
2022-03-11    861509.779196
2022-03-12    861584.451114
2022-03-13    861659.123031
2022-03-14    861733.794948
2022-03-15    861808.466865
2022-03-16    861883.138783
2022-03-17    861957.810700
2022-03-18    862032.482617
2022-03-19    862107.154534
2022-03-20    862181.826452
2022-03-21    862256.498369
2022-03-22    862331.170286
2022-03-23    862405.842203
2022-03-24    862480.514121
2022-03-25    862555.186038
2022-03-26    862629.857955
2022-03-27    862704.529872
2022-03-28    862779.201790
2022-03-29    862853.873707
2022-03-30    862928.545624
2022-03-31    863003.217541
Freq: D, dtype: float64

In [24]:
y_test

2022-03-08    861323.0
2022-03-09    861468.0
2022-03-10    861617.0
2022-03-11    861733.0
2022-03-12    861879.0
2022-03-13    862044.0
2022-03-14    862098.0
2022-03-15    862226.0
2022-03-16    862355.0
2022-03-17    862509.0
2022-03-18    862660.0
2022-03-19    862812.0
2022-03-20    862926.0
2022-03-21    862978.0
2022-03-22    863055.0
2022-03-23    863157.0
2022-03-24    863223.0
2022-03-25    863319.0
2022-03-26    863427.0
2022-03-27    863538.0
2022-03-28    863576.0
2022-03-29    863623.0
2022-03-30    863704.0
2022-03-31    863790.0
Freq: D, Name: 12-17yrs, dtype: float64